In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np

default_dir = os.path.dirname(os.getcwd())
os.chdir(default_dir)

import RLConn as rc

W0410 11:30:17.712868 4720741824 deprecation_wrapper.py:119] From /Users/Frank/github/RLConn/RLConn/control_dqn.py:5: The name tf.set_random_seed is deprecated. Please use tf.compat.v1.set_random_seed instead.

W0410 11:30:17.732421 4720741824 deprecation_wrapper.py:119] From /Users/Frank/github/RLConn/RLConn/control_dqn.py:58: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0410 11:30:17.740344 4720741824 deprecation_wrapper.py:119] From /Users/Frank/github/RLConn/RLConn/control_dqn.py:66: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0410 11:30:17.741391 4720741824 deprecation.py:323] From /Users/Frank/github/RLConn/RLConn/control_dqn.py:68: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.
W0410 11:30:18.165222 4720741824 deprecation_wrapper.py:119] From /Users/Frank/github/RLConn/RLConn/control

In [2]:
def get_random_connectome(N):
    l = int(N*(N-1)/2)
    Gg = np.zeros((N,N))
    Gg[np.logical_not(np.tri(N,dtype=bool))] = np.random.randint(0,10,l)
    Gg += Gg.T
    Gs = np.zeros((N,N))
    Gs[np.tri(N,dtype=bool,k=-1)] = np.random.rand(l)
    Gs[np.logical_not(np.tri(N,dtype=bool))] = np.random.randint(0,10,l)
    return Gg, Gs

In [3]:
def get_observe(Gg, Gs):
    N = Gg.shape[0]
    return np.concatenate((Gg[np.triu_indices(N,k=1)],Gs[np.triu_indices(N,k=1)],Gs[np.tril_indices(N,k=-1)]))

In [4]:
def move(Gg, Gs, action, location, dx = 0.1):
    N = Gg.shape[0]
    size = int(N*(N-1)/2)
    change = dx*(action == 0) + (-dx)*(action == 2)
    if location < size:
        coordinate = np.array(np.triu_indices(N,k=1)).T[location]
        Gg[coordinate[0], coordinate[1]] += change
        Gg[coordinate[1], coordinate[0]] += change
    elif location < 2*size:
        coordinate = np.array(np.triu_indices(N,k=1)).T[location - size]
        Gs[coordinate[0], coordinate[1]] += change
    else: 
        coordinate = np.array(np.tril_indices(N,k=-1)).T[location - 2*size]
        Gs[coordinate[0], coordinate[1]] += change
    return Gg, Gs

In [5]:
def get_rand_action(N):
    size = int(N*(N-1)/2)
    return np.random.randint(2), np.random.randint(size)

In [6]:
def update_x(x, Gg, Gs, reward):
    obs = get_observe(Gg, Gs)
    x = np.concatenate((x[len(obs)+1:], obs))
    x = np.append(x,reward)
    return x

In [7]:
# initialize
N = 3
size = int(3*N*(N-1)/2)
PG = rc.alpg.ActLocPolicyGradient(
    n_actions=3,
    n_features=(size+1)*10)

In [8]:
for i_episode in range(3000):
    #initialize x
    x = np.array([])
    Gg, Gs = get_random_connectome(N)
    reward = -rc.utils.scorer(Gg, Gs)
    x = np.concatenate((x, get_observe(Gg, Gs)))
    x = np.append(x,reward)
    for i in range(9):
        action, location = get_rand_action(N)
        Gg, Gs = move(Gg, Gs, action, location, dx=0.1)
        reward = -rc.utils.scorer(Gg, Gs)
        x = np.concatenate((x, get_observe(Gg, Gs)))
        x = np.append(x,reward)   
    print(len(x))
    
    #choose action using nn
    for step in range(300):
        action, location = PG.choose_action(x)
        Gg, Gs = move(Gg, Gs, action, location, dx=0.1)
        reward = -rc.utils.scorer(Gg, Gs)
        PG.store_transition(x, action, location, reward)
        x = update_x(x, Gg, Gs, reward)
        print(len(x))
    ep_rs_sum = sum(RL.ep_rs)
    if 'running_reward' not in globals():
        running_reward = ep_rs_sum
    else:
        running_reward = running_reward * 0.99 + ep_rs_sum * 0.01
    print("episode:", i_episode, "  reward:", int(running_reward))
    print("Gg:", Gg.flatten(), "  Gs:", Gs.flatten())
    vt = PG.learn()
    # print(vt)

AttributeError: module 'RLConn.utils' has no attribute 'scorer'